In [13]:
!rm -r mom_rag

In [14]:
!git clone https://github.com/Francesco9932/mom_rag

Cloning into 'mom_rag'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 125 (delta 65), reused 31 (delta 8), pack-reused 0
Receiving objects: 100% (125/125), 2.04 MiB | 4.92 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
!sudo apt install build-essential
!sudo apt install gcc-11 g++-11

In [2]:
!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 38.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.84-cp310-cp310-linux_x86_64.whl size=255059329 sha256=c5c64e07f64895dc1e7f93585e7161a815caf35bc83d9a2cc0b493a42ba88f41
  Stored in directory: /root/.cache/pip/wheels/b1/e8/4d/99139e9d4d14209b822d903fe458fd962cc4a902cbfd78578d
Successfully built llama-cpp-python


In [3]:
from llama_cpp import Llama

In [ ]:
!pip install -q langchain
!pip install -U langchain-community
!pip install -q torch
!pip install -q transformers==4.41.2
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install tiktoken
!pip install einops
!pip install flash-attn --no-build-isolatioz

In [4]:
with open("./mom_rag/transcript_EUROPARL.txt", 'r', encoding="utf-8") as f:
  data = f.read()

In [9]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF", filename="Meta-Llama-3.1-8B-Instruct-Q8_0.gguf", local_dir="llama3.1")

Meta-Llama-3.1-8B-Instruct-f32.gguf:   0%|          | 0.00/32.1G [00:00<?, ?B/s]

'llama3.1/Meta-Llama-3.1-8B-Instruct-f32.gguf'

In [17]:
from llama_cpp import Llama
import os
import torch

prompt="""<|system|>

Sei un assistente di una azienda che è specializzato nel riassumere meeting in maniera precisa.
Utilizza le seguenti regole:
1. Bullet Points per effettuare il riassunto;
2. Lingua di output: ITALIANO.
<|end|>
<|user|>
Meeting Text: "{text}"
RIASSUNTO: <|end|>
<|assistant|>"""

MODEL_NAME = "./llama3.1/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"

print(f"Loading Summary LLM")
summary_llm = Llama(
        model_path=MODEL_NAME,
        n_gpu_layers = -1,
        n_ctx=14000,
        verbose=True
    )
print(f"Loaded Summary LLM")

print(f"Generating MoM")
output = summary_llm(
    prompt.format(text=data),
    stop = ["<|end|>"],
    max_tokens = 1000
)
del summary_llm
torch.cuda.empty_cache()
print(f"Finished Generating MoM")
print(output["choices"][0]["text"])

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from ./llama3.1/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = llama3.1
llama_model_loader: - kv  

Loading Summary LLM


llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.7999 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 131072
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1

Loaded Summary LLM
Generating MoM



llama_print_timings:        load time =     201.72 ms
llama_print_timings:      sample time =      56.37 ms /   561 runs   (    0.10 ms per token,  9952.63 tokens per second)
llama_print_timings: prompt eval time =    5218.41 ms /  6889 tokens (    0.76 ms per token,  1320.13 tokens per second)
llama_print_timings:        eval time =   22826.85 ms /   560 runs   (   40.76 ms per token,    24.53 tokens per second)
llama_print_timings:       total time =   29222.59 ms /  7449 tokens


Finished Generating MoM
 RIASSUNTO DEL MEETING:

* **Sostituzione del meeting:** [PERSON54] ha proposto di sostituire il meeting con una sessione di lavoro online tramite Zoom, in modo da poter discutere e lavorare in modo più efficiente.
* **Creazione del link di Zoom:** [PERSON27] ha creato il link di Zoom per la sessione di lavoro e lo ha condiviso con il team.
* **Uso del link:** [PERSON54] ha spiegato che il link di Zoom è lo stesso per tutte le sessioni di lavoro e che può essere utilizzato per tutte le riunioni di coffee meeting.
* **Distribuzione dei documenti:** [PERSON54] ha proposto di distribuire i documenti tramite Google Drive, anziché tramite email.
* **Problemi con Google Meet:** [PERSON27] ha menzionato che Google Meet non funziona più gratuitamente per più di un'ora, quindi potrebbe essere necessario pagare per utilizzarlo.
* **Riassunto delle attività:** [PERSON54] ha chiesto a [PERSON27] di riassumere le attività da svolgere per il progetto, inclusa la creazione dei